In [11]:
import numpy
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time

In [2]:
# Función para extraer datos de una página
def scrape_pagina(url):
    # Realizar la solicitud HTTP
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    if response.ok:
        print("La solicitud fue exitosa.")
    else:
        print(f"La solicitud falló con el código de estado {response.status_code}.")
    # Crear el objeto BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar todos los divs con la clase "posting-container"
    posteos = soup.find_all('div', class_='CardContainer-sc-1tt2vbg-5 fvuHxG')
    return posteos
       

In [37]:
posteos=scrape_pagina('link')

La solicitud fue exitosa.


In [3]:
def procesar_precio(precio_text):
    # Eliminar espacios en blanco y convertir a minúsculas
    precio_text = precio_text.strip().lower()

    # Extraer el número del texto
    numero = re.sub(r'[^\d]', '', precio_text)

    if not numero:
        return None, None  # Si no se encuentra un número, retornar None

    # Convertir a float
    cantidad = float(numero)

    # Determinar la moneda
    if '$' in precio_text and 'usd' not in precio_text:
        moneda = 'pesos'
    elif 'usd' in precio_text:
        moneda = 'dolares'
    else:
        moneda = 'desconocido'

    return cantidad, moneda

In [4]:
def extraer_caracteristicas(caracteristicas_crudas):
    caracteristicas = {}
    if caracteristicas_crudas:
        spans = caracteristicas_crudas.find_all('span')
        for span in spans:
            text = span.text.strip().lower()
            if 'm²' in text:
                caracteristicas['area'] = int(text.split()[0])
            elif 'amb.' in text:
                caracteristicas['ambientes'] = int(text.split()[0])
            elif 'dorm.' in text:
                caracteristicas['dormitorios'] = int(text.split()[0])
            elif 'baño' in text:
                caracteristicas['baños'] = int(text.split()[0])
            elif 'coch.' in text:
                caracteristicas['cocheras'] = int(text.split()[0])
    return caracteristicas

In [5]:
def analizar_caracteristicas(caracteristicas_str):
    caracteristicas = {}

    # Superficie
    sup = re.search(r'(\d+)\s*m²', caracteristicas_str)
    caracteristicas['superficie (m2)'] = int(sup.group(1)) if sup else 0

    # Ambientes totales
    amb = re.search(r'(\d+)\s*amb', caracteristicas_str)
    caracteristicas['ambientes totales'] = int(amb.group(1)) if amb else 0

    # Dormitorios
    dor = re.search(r'(\d+)\s*dorm', caracteristicas_str)
    caracteristicas['dormitorios'] = int(dor.group(1)) if dor else 0

    # Baños
    baños = re.search(r'(\d+)\s*baño', caracteristicas_str)
    caracteristicas['baños'] = int(baños.group(1)) if baños else 0

    # Cocheras
    coch = re.search(r'(\d+)\s*coch', caracteristicas_str)
    caracteristicas['cocheras'] = int(coch.group(1)) if coch else 0

    return caracteristicas

In [6]:
analizar_caracteristicas('32 m² tot.2 amb.1 dorm.1 baño')

{'superficie (m2)': 32,
 'ambientes totales': 2,
 'dormitorios': 1,
 'baños': 1,
 'cocheras': 0}

In [7]:
def purgar_barrio(barrio):
  barrios_oficiales=['Agronomía','Almagro','Balvanera','Barracas','Belgrano','Boedo','Caballito',
                     'Chacarita','Coghlan','Colegiales','Constitución','Congreso','Flores','Floresta','La Boca',
                     'La Paternal','Liniers','Mataderos','Monte Castro','Monserrat','Nueva Pompeya',
                     'Núñez','Palermo','Parque Avellaneda','Parque Chacabuco','Parque Chas','Parque Patricios',
                     'Puerto Madero','Recoleta','Retiro','Saavedra','San Cristóbal','San Nicolás','San Telmo',
                     'Vélez Sársfield','Versalles','Villa Crespo','Villa del Parque','Villa Devoto','Villa General Mitre',
                     'Villa Lugano','Villa Luro','Villa Ortúzar','Villa Pueyrredón','Villa Real','Villa Riachuelo',
                     'Villa Santa Rita','Villa Soldati','Villa Urquiza']
  partes = barrio.split(',')
  res=partes[0]
  for parte in partes:
    parte=parte.strip()
    if parte in barrios_oficiales:
      res = parte
      break
  return res

In [105]:
# Lista para almacenar los datos extraídos
data = []
#df = pd.DataFrame(data)

# Función para extraer datos de una página
def procesar(posteos):

    for posteo in posteos:
        # Extraer información de cada posteo
        precio_text = posteo.find('div', class_='Price-sc-12dh9kl-3 geYYII').text if posteo.find('div', class_='Price-sc-12dh9kl-3 geYYII') else 'N/A'

        # Procesar el precio
        precio, moneda = procesar_precio(precio_text)

        expensas = posteo.find('div', class_='Expenses-sc-12dh9kl-1 iboaIF').text if posteo.find('div', class_='Expenses-sc-12dh9kl-1 iboaIF') else 'N/A'

        #procesar expensas
        precio_expensas, moneda_expensas = procesar_precio(expensas)

        barrio_crudo = posteo.find('h2', class_='LocationLocation-sc-ge2uzh-2 fziprF').text if posteo.find('h2', class_='LocationLocation-sc-ge2uzh-2 fziprF').text else 'N/A'
        #procesar barrios
        barrio= purgar_barrio(barrio_crudo)

        caracteristicas_crudo = posteo.find('h3', class_='PostingMainFeaturesBlock-sc-1uhtbxc-0 cHDgeO').text if posteo.find('h3', class_='PostingMainFeaturesBlock-sc-1uhtbxc-0 cHDgeO') else 'N/A'
        #procesar carateristicas
        caracteristicas = analizar_caracteristicas(caracteristicas_crudo)
        #buscar si es dueño directo
        descripcion = posteo.find('a').text.lower()
    
        # Verifica si las palabras "inmobiliaria" o "inmobiliario" o mencion a "dueño directo" están en el texto
        if descripcion:
            
            if 'inmobiliaria' or 'inmobiliario' in descripcion:
                arrendador = 'Inmobiliaria'
            
            elif 'dueño directo' or 'dueño-directo' or 'directo' in descripcion:
                arrendador = 'Dueño directo'
        
        else:
            arrendedor = 'Indefinido'

        # Añadir los datos extraídos a la lista
        data.append({
            'Precio': precio,
            'Moneda': moneda,
            'Expensas': precio_expensas,
            'Moneda Expensas': moneda_expensas, 
            'Barrio': barrio,
            'Arrendador':arrendador,
            'Superficie (m2)': caracteristicas['superficie (m2)'],
            'Ambientes': caracteristicas['ambientes totales'],
            'Dorm': caracteristicas['dormitorios'],
            'Baños': caracteristicas['baños'],
            'Cocheras': caracteristicas['cocheras']
        })
    
    print('script competado')
    return
# Crear un DataFrame con los datos extraídos

In [96]:
procesar(posteos)

script competado


In [97]:
data[0]

{'Precio': 1000.0,
 'Moneda': 'dolares',
 'Expensas': 160000.0,
 'Moneda Expensas': 'pesos',
 'Barrio': 'Belgrano',
 'Arrendador': 'Inmobiliaria',
 'Superficie (m2)': 95,
 'Ambientes': 4,
 'Dorm': 3,
 'Baños': 2,
 'Cocheras': 0}

In [22]:
soup = posteos[0]

# Extraer información de los divs con las clases especificadas
price = soup.find('div', class_='Price-sc-12dh9kl-3 geYYII').text
precio = procesar_precio(price)
expensas_aux = soup.find('div', class_='Expenses-sc-12dh9kl-1 iboaIF').text
expensas = procesar_precio(expensas_aux)
location = soup.find('h2', class_='LocationLocation-sc-ge2uzh-2 fziprF').text
localidad = purgar_barrio(location)
features = soup.find('h3', class_='PostingMainFeaturesBlock-sc-1uhtbxc-0 cHDgeO').text
carac = analizar_caracteristicas(features)


# Imprimir los valores extraídos
print("Price:", price , precio)
print("Expenses:",expensas_aux, expensas)
print("Location:", location, localidad)
print("Main Features:", features)
print(carac)
print(type(carac))

Price: USD 2.200 (2200.0, 'dolares')
Expenses: $ 550.000 Expensas (550000.0, 'pesos')
Location: Belgrano, Capital Federal Belgrano
Main Features: 135 m² tot.3 amb.2 dorm.4 baños
{'superficie (m2)': 135, 'ambientes totales': 3, 'dormitorios': 2, 'baños': 4, 'cocheras': 0}
<class 'dict'>


In [118]:
def realizar_scraping(url_base, total_paginas):
    for i in range(1, total_paginas + 1):
        url = f"{url_base}-pagina-{i}.html"
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                 # Crear el objeto BeautifulSoup
                soup = BeautifulSoup(response.text, 'html.parser')

                # Encontrar todos los divs con la clase "posting-container"
                posteos = soup.find_all('div', class_='CardContainer-sc-1tt2vbg-5 fvuHxG')
                print(f"Página {i} OK")
                
                procesar(posteos)
            else:
                print(f"Error en la página {i}: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Solicitud fallida en la página {i}: {e}")
        
        # Tiempo de espera entre solicitudes
        time.sleep(3)

In [ ]:
realizar_scraping('link',333)

In [ ]:
#el codifo es ejecutado correctamente por lo que la infomacion queda recopilada en data
# a continuacion creo el dataset

In [120]:
len(data)

3780

In [121]:
df = pd.DataFrame(data)

In [122]:
df

,Precio,Moneda,Expensas,Moneda Expensas,Barrio,Arrendador,Superficie (m2),Ambientes,Dorm,Baños,Cocheras
0,1200.0,dolares,NaN,None,Recoleta,Inmobiliaria,125,4,3,3,0
1,900.0,dolares,137000.0,pesos,Palermo,Inmobiliaria,42,2,1,1,1
2,1000.0,dolares,NaN,None,Palermo,Inmobiliaria,81,3,2,1,0
3,2500.0,dolares,NaN,None,Palermo,Inmobiliaria,81,3,2,2,1
4,350.0,dolares,216000.0,pesos,Palermo,Inmobiliaria,33,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
3775,450000.0,pesos,105000.0,pesos,Almagro,Inmobiliaria,53,3,2,1,0
3776,500000.0,pesos,55000.0,pesos,Flores,Inmobiliaria,118,2,1,1,0
3777,550.0,dolares,180000.0,pesos,Recoleta,Inmobiliaria,42,1,0,1,0
3778,600.0,dolares,280000.0,pesos,Belgrano,Inmobiliaria,55,2,1,1,0


In [134]:
df.to_csv('alquileresDB.csv', index = False)
df.to_excel('alquileresDB.xlsx', index = False)
df.to_json('alquileresDB.json')